#                                        BrowserID Fanout

In [ ]:
print('Please wait. Installing required packages. This may take a few minutes...')
!pip install Kqlmagic --no-cache-dir --upgrade

<a id='connect-to-la'></a>
## Connect to Kusto Database
This is required to read the tables in your kusto data source. 

In [ ]:
%reload_ext Kqlmagic
%kql azure_data-Explorer://code;cluster='estsch1';database='ESTS'

<a id='user-input'></a>
## Enter Tenant Information 

In [ ]:
from datetime import date
import ipywidgets as widgets
from IPython.display import display
import pandas as pd  
from pandas import ExcelWriter

print('Enter Tenant Id [Format=> "<tenant-id>", multiple-tenant-id: "<tenant-id1>","<tenant-id2>"]:')
TenantId = input().strip()
print()

print("Enter total days to retrive ESTS logs, It sholud not be more than 60:")
total_days = input().strip()
print()
if int(total_days) > 60:
    print("Enter total days to retrive ESTS logs, It sholud not be more than 60:")
    total_days = input().strip()
    print()
if not TenantId or not total_days:
    raise Exception("Error: tenant id and days could not be empty.")

This kusto query will divide total days by 5 and run for 5-5 days. After completion of below cell , you will see "browserid_list.csv" file inside your project which is containing result of below kusto query

In [ ]:
days=0
ex_days=int(total_days)%5
remain_days=int(total_days)
i=5
startdays=0
while days < int(total_days):
    print("inside for loop "+str(days)+"d , adding"+str(i)+"d")
    try:
        startdays=int(total_days) if int(total_days)<5 else days+i
        enddays=days
        br_query=f"""
        set notruncation;
        set query_optimization_costbased_enabled;
        let StartDate = ago({startdays}d);
        let EndDate =ago({enddays}d);
        let tenant_set=datatable(TenantId:string)[{TenantId}];
        let Brow_ids=pack_array("");
        let Query=(clusterName:string) {{
            cluster(clusterName).database("ESTS").PerRequestTableIfx
                | where env_time between (StartDate .. EndDate) 
                //| where BrowserId  in (Brow_ids)
                | extend validLogin=iif( RamAuthenticationStatus == long(1), "True", iif(AuthType == 1 and AuthMethods > 0, "True", "False"))
                | where validLogin==True
                | where TenantId in (tenant_set)
                |project BrowserId, TenantId,DomainName,UserPrincipalObjectID,ClientIp, RamAccountRiskRecommendedAction,RamSessionRiskRecommendedAction,PUID, UserAgent,validLogin  
            }};
        (Query("estsch1")) | union (Query("estsby1")),(Query("estssn1")),(Query("estsbl2")),(Query("estsam2")),(Query("estsdb3")),(Query("estshkn")),(Query("estssin")),(Query("estsause")),(Query("estsdm1")),(Query("estskrs")),(Query("estswus2"))
        """
        #print(br_query)
        %kql PT_frm << {br_query}
        PT_frm = PT_frm.to_dataframe()
        PT_frm.to_csv("browserid_list.csv", mode='a',header=False,index=False)
        remain_days=remain_days-days
        days=days+5 
        i=5 if remain_days%5 == 0 else ex_days
    except Exception as ex:
        print("exception",ex)

In [ ]:
import json
br_data = pd.read_csv("browserid_list.csv")
br_data=br_data.drop_duplicates(subset=None, keep='first', inplace=False)
br_data.columns = ["browser id", "TenantId","DomainName","UserPrincipalObjectID","ClientIp","RamAccountRiskRecommendedAction","RamSessionRiskRecommendedAction","PUID","UserAgent","validLogin"]
first_br_list=ip_list=list(dict.fromkeys(br_data['browser id']))
first_br_list

This step for finding associated tenant with browser id which found from above kusto query

In [ ]:
days=0
ex_days=int(total_days)%5
remain_days=int(total_days)
i=5
startdays=0
while days < int(total_days):
    print("inside for loop "+str(days)+"d , adding"+str(i)+"d")
    try:
        startdays=int(total_days) if int(total_days)<5 else days+i
        enddays=days
        br_query=f"""
        set notruncation;
        set query_optimization_costbased_enabled;
        let StartDate = ago({startdays}d);
        let EndDate =ago({enddays}d);
        let tenant_set=datatable(TenantId:string)[{TenantId}];
        let Brow_ids=datatable(BrowserId:string){first_br_list};
        let Query=(clusterName:string) {{
            cluster(clusterName).database("ESTS").PerRequestTableIfx
                |where env_time between (StartDate .. EndDate) 
                |extend validLogin=iif( RamAuthenticationStatus == long(1), "True", iif(AuthType == 1 and AuthMethods > 0, "True", "False"))
                //|where validLogin==True
                |where TenantId !in (tenant_set)
                //|where isnotempty(BrowserId)
                |where BrowserId in (Brow_ids)
                |where isnotempty(TenantId)
                |distinct BrowserId,TenantId,DomainName,UserPrincipalObjectID,ClientIp, RamAccountRiskRecommendedAction,RamSessionRiskRecommendedAction,PUID, UserAgent,validLogin
            }};
        (Query("estsch1")) | union (Query("estsby1")),(Query("estssn1")),(Query("estsbl2")),(Query("estsam2")),(Query("estsdb3")),(Query("estshkn")),(Query("estssin")),(Query("estsause")),(Query("estsdm1")),(Query("estskrs")),(Query("estswus2"))
        """
        #print(br_query)
        %kql PT_frm << {br_query}
        PT_frm = PT_frm.to_dataframe()
        PT_frm.to_csv("tenantid_list.csv", mode='a',header=False,index=False)
        remain_days=remain_days-days
        days=days+5 
        i=5 if remain_days%5 == 0 else ex_days
    except Exception as ex:
        print("exception",ex)

In [ ]:
te_data = pd.read_csv("tenantid_list.csv")
te_data=te_data.drop_duplicates(subset=None, keep='first', inplace=False)
te_data.columns = ['BrowserId','TenantId','DomainName','UserPrincipalObjectID','ClientIp', 'RamAccountRiskRecommendedAction','RamSessionRiskRecommendedAction','PUID', 'UserAgent','validLogin']

In [ ]:
te_data.to_csv("tenantid_list.csv") ## this is Tenant list where we need to perform Real/Fake tenant analysis 

#### "tenantid_list.csv" is Tenant list where we need to perform Real/Fake tenant analysis 

### [Under-Development] Move Integration of Neo-4j to Azure notebook